# Ball Tracking
### Team Catan 

##### Read and show the video file

In [11]:
import cv2
import glob

In [12]:
# give the directory and the px size of the video files
# of the recordings you want to show and track
recording_dir = 'recordings40px025/'
# give the pixel size of the video files
px_size = 128

In [13]:

mp4_files = glob.glob(recording_dir + '*.mp4')
cap = None
if not mp4_files:
    print('No mp4 files found')
else:
    first_file = mp4_files[0]
    cap = cv2.VideoCapture(first_file)

if not cap.isOpened():
    print("Error opening video stream or file")

else:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        cv2.imshow('Frame', frame)
        if cv2.waitKey(25) & 0xFF == ord('q'): # Press q to exit
            break
            
cap.release()
cv2.destroyAllWindows()

No mp4 files found


AttributeError: 'NoneType' object has no attribute 'isOpened'

#### Detect the red ball and show

In [ ]:
import numpy as np

cap = cv2.VideoCapture(mp4_files[0])

detected_ball = None
last_frame = None
lower_red = np.array([0, 0, 100], dtype=np.uint8)
upper_red = np.array([75, 75, 255], dtype=np.uint8)


while cap.isOpened():
    # Read a frame from the video
    ret, frame = cap.read()

    # Check if the frame was read successfully
    if not ret:
        break
    # Create a mask to isolate the red color
    mask = cv2.inRange(frame, lower_red, upper_red)
    # Apply the mask to the frame to detect the red color
    detected_ball = cv2.bitwise_and(frame, frame, mask=mask)
    #   cv2.imshow('Current Frame', frame)
    cv2.imshow('Detected Ball', detected_ball)
    last_frame = frame
    if cv2.waitKey(25) & 0xFF == ord('q'): # Press q to exit
        break

cap.release()
cv2.destroyAllWindows()

#### Show the amount of frames in the video

In [ ]:
cap = cv2.VideoCapture(mp4_files[0])
length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
length

#### Get and Show the coordinates of the red ball in one frame

In [ ]:
# get all the coordinates of the pixels that are between the lower and upper red
red_coordinates = np.column_stack(np.where((last_frame[:, :, 0] >= lower_red[0]) & (last_frame[:, :, 0] <= upper_red[0]) &
    (last_frame[:, :, 1] >= lower_red[1]) & (last_frame[:, :, 1] <= upper_red[1]) &
    (last_frame[:, :, 2] >= lower_red[2]) & (last_frame[:, :, 2] <= upper_red[2])
))
# invert the y-axis so that the origin is at the bottom left
red_coordinates[:, 0] = last_frame.shape[0] - red_coordinates[:, 0]

In [ ]:
from matplotlib import pyplot as plt
plt.scatter(red_coordinates[:, 1], red_coordinates[:, 0], c='red', marker='.')
plt.title('Red Ball Coordinates')
plt.xlabel('X Coordinates')
plt.ylabel('Y Coordinates')
plt.xlim(0, px_size)
plt.ylim(0, px_size)
plt.show()


In [ ]:
# calculate the mean of the x and y coordinates
mean_x = np.mean(red_coordinates[:, 1])
mean_y = np.mean(red_coordinates[:, 0])
mean_x, mean_y

#### Get the middle coordinate of the red ball in all frames

In [ ]:
from ball_tracker_calculations import calculate_middle_points_of_ball
cap = cv2.VideoCapture(mp4_files[0])

lower_red = np.array([0, 0, 100], dtype=np.uint8)
upper_red = np.array([75, 75, 255], dtype=np.uint8)
mean_x, mean_y = calculate_middle_points_of_ball(cap, lower_red, upper_red)

### Show the trajectory of the ball

In [ ]:
from plot_maker import create_plot_of_trajectory

create_plot_of_trajectory(mean_x, mean_y, px_size)